In [1]:
from controllables.core import TemporaryUnavailableError
from controllables.energyplus import (
    World,
    #WeatherModel,
    #Report,
    Actuator,
    OutputVariable,
)

from energyplus.dataset.basic import dataset as _epds_

world = World(
    input=World.Specs.Input(
        world=(
            _epds_.models / '1ZoneEvapCooler.idf'
        ),
        weather=(_epds_.weathers / 'USA_CO_Denver-Aurora-Buckley.AFB.724695_TMY3.epw'),
    ),
    output=World.Specs.Output(
        #report=('/tmp/ooep-report-9e1287d2-8e75-4cf5-bbc5-f76580b56a69'),
    ),
    runtime=World.Specs.Runtime(
        #recurring=True,
        #design_day=True,
    ),
)
world.add('logging:progress')

  0%|          | 0/100 [00:00<?, ?it/s]

### Barebone

In [2]:
from controllables.core.tools.gymnasium import (
    Agent,
    DictSpace,
    BoxSpace,
)

import numpy as _numpy_


agent = Agent({
    'action_space': DictSpace({
        'thermostat': BoxSpace(
            low=15., high=20.,
            dtype=_numpy_.float32,
            shape=(),
        ).bind(Actuator.Ref(
            type='Zone Temperature Control',
            control_type='Heating Setpoint',
            key='MAIN ZONE',
        ))
    }),    
    'observation_space': DictSpace({
        'temperature': BoxSpace(
            low=-_numpy_.inf, high=+_numpy_.inf,
            dtype=_numpy_.float32,
            shape=(),
        ).bind(world[OutputVariable.Ref(
            type='Zone Air Temperature',
            key='MAIN ZONE',
        )]),
    }),
}).__attach__(world)

In [3]:
world.start()

In [4]:
for _ in range(10):
    ctx = world.events['end_zone_timestep_after_zone_reporting'].wait()
    print(ctx)

ExecutionContext(vars=Arguments(Context(event=<controllables.energyplus.events.Event object at 0x7fdb9034b6d0>), ), ack=<controllables.core.callables.ExecutionContext.Ack object at 0x7fdb903476d0>)
ExecutionContext(vars=Arguments(Context(event=<controllables.energyplus.events.Event object at 0x7fdb9034b6d0>), ), ack=<controllables.core.callables.ExecutionContext.Ack object at 0x7fdb90353610>)
ExecutionContext(vars=Arguments(Context(event=<controllables.energyplus.events.Event object at 0x7fdb9034b6d0>), ), ack=<controllables.core.callables.ExecutionContext.Ack object at 0x7fdb90350a50>)
ExecutionContext(vars=Arguments(Context(event=<controllables.energyplus.events.Event object at 0x7fdb9034b6d0>), ), ack=<controllables.core.callables.ExecutionContext.Ack object at 0x7fdb90352c10>)
ExecutionContext(vars=Arguments(Context(event=<controllables.energyplus.events.Event object at 0x7fdb9034b6d0>), ), ack=<controllables.core.callables.ExecutionContext.Ack object at 0x7fdb90350110>)
ExecutionC

In [5]:
for _ in range(100):
    ctx = await world.events['end_zone_timestep_after_zone_reporting'].awaitable(deferred=True)
    try: 
        print(world['wallclock'].value, agent.observe())
    except TemporaryUnavailableError: 
        pass
    except Exception as e: 
        ctx.ack()
        raise e
    agent.act({'thermostat': 20})
    ctx.ack()

2005-01-01 00:10:00 {'temperature': array(20., dtype=float32)}
2005-01-01 00:20:00 {'temperature': array(20., dtype=float32)}
2005-01-01 00:30:00 {'temperature': array(20., dtype=float32)}
2005-01-01 00:40:00 {'temperature': array(20., dtype=float32)}
2005-01-01 00:50:00 {'temperature': array(20., dtype=float32)}
2005-01-01 01:00:00 {'temperature': array(20., dtype=float32)}
2005-01-01 01:10:00 {'temperature': array(20., dtype=float32)}
2005-01-01 01:20:00 {'temperature': array(20., dtype=float32)}
2005-01-01 01:30:00 {'temperature': array(20., dtype=float32)}
2005-01-01 01:40:00 {'temperature': array(20., dtype=float32)}
2005-01-01 01:50:00 {'temperature': array(20., dtype=float32)}
2005-01-01 02:00:00 {'temperature': array(20., dtype=float32)}
2005-01-01 02:10:00 {'temperature': array(20., dtype=float32)}
2005-01-01 02:20:00 {'temperature': array(20., dtype=float32)}
2005-01-01 02:30:00 {'temperature': array(20., dtype=float32)}
2005-01-01 02:40:00 {'temperature': array(20., dtype=fl

In [6]:
world.stop()

### `gymnasium` `Env`s

In [7]:
from controllables.core.tools.gymnasium import Env

env = Env({
    'action_space': DictSpace({
        'thermostat': BoxSpace(
            low=15., high=20.,
            dtype=_numpy_.float32,
            shape=(),
        ).bind(Actuator.Ref(
            type='Zone Temperature Control',
            control_type='Heating Setpoint',
            key='MAIN ZONE',
        ))
    }),    
    'observation_space': DictSpace({
        'temperature': BoxSpace(
            low=-_numpy_.inf, high=+_numpy_.inf,
            dtype=_numpy_.float32,
            shape=(),
        ).bind(world[OutputVariable.Ref(
            type='Zone Air Temperature',
            key='MAIN ZONE',
        )]),
    }),
    'reward_function': lambda agent: 1,
    'events': {'step': 'end_zone_timestep_after_zone_reporting'},
}).__attach__(world)

In [8]:
world.start()

In [9]:
for _ in range(100):
    try: print('step', env.step(env.action_space.sample()))
    except TemporaryUnavailableError: pass

step StepResult(observation={'temperature': array(17.182953, dtype=float32)}, reward=1, terminated=False, truncated=False, info={})
step StepResult(observation={'temperature': array(17.701578, dtype=float32)}, reward=1, terminated=False, truncated=False, info={})
step StepResult(observation={'temperature': array(18.159428, dtype=float32)}, reward=1, terminated=False, truncated=False, info={})
step StepResult(observation={'temperature': array(17.0818, dtype=float32)}, reward=1, terminated=False, truncated=False, info={})
step StepResult(observation={'temperature': array(16.666447, dtype=float32)}, reward=1, terminated=False, truncated=False, info={})
step StepResult(observation={'temperature': array(17.443327, dtype=float32)}, reward=1, terminated=False, truncated=False, info={})
step StepResult(observation={'temperature': array(18.005993, dtype=float32)}, reward=1, terminated=False, truncated=False, info={})
step StepResult(observation={'temperature': array(18.319244, dtype=float32)}, 

step StepResult(observation={'temperature': array(17.736328, dtype=float32)}, reward=1, terminated=False, truncated=False, info={})
step StepResult(observation={'temperature': array(17.872324, dtype=float32)}, reward=1, terminated=False, truncated=False, info={})
step StepResult(observation={'temperature': array(16.24145, dtype=float32)}, reward=1, terminated=False, truncated=False, info={})
step StepResult(observation={'temperature': array(17.13417, dtype=float32)}, reward=1, terminated=False, truncated=False, info={})
step StepResult(observation={'temperature': array(17.482183, dtype=float32)}, reward=1, terminated=False, truncated=False, info={})
step StepResult(observation={'temperature': array(16.499548, dtype=float32)}, reward=1, terminated=False, truncated=False, info={})
step StepResult(observation={'temperature': array(16.347603, dtype=float32)}, reward=1, terminated=False, truncated=False, info={})
step StepResult(observation={'temperature': array(16.659033, dtype=float32)}, 

In [10]:
world.stop()